In [1]:
import psycopg2 as pg
import pandas as pd
import re

In [2]:
def create_db_connection(host_name, user_name, user_password, db_name):
    conn = None
    try:
        conn = pg.connect(
            host=host_name,
            user = user_name,
            password = user_password,
            database = db_name
        )
        print("PostgreSQL '{}' Database connection successful".format(db_name))
    except Exception as e:
        print(f"Error: '{e}'")
    return conn

def execute_query(q, conn):
    cur = conn.cursor()
    try:
        cur.execute(q)
        conn.commit()
        print('Query successful!\n')
    except Exception as e:
        print(f"Error: '{e}'")
        conn.rollback()

In [3]:
db = 'datacampsales'
csvpath = 'data/traffic.csv'
table_name = 'traffic'

df = pd.read_csv(csvpath)

In [4]:
conn = create_db_connection('localhost', 'postgres', 'password', db)
cur=conn.cursor()

PostgreSQL 'datacampsales' Database connection successful


In [5]:
def create_schema_from_df(data, table_name):
    dtype_map = {
        'float64': 'float',
        'int64': 'bigint',
        'bool': 'boolean',
        'datetime64': 'timestamp',
        'timedelta64[ns]': 'varchar',
        'object': 'varchar',
    }

    schema_list = []
    
    for column_name, dtype in data.dtypes.items():
        column_name = re.sub('[^A-Za-z0-9]+', '_', column_name).lower()
        column_name = str.strip(column_name)

        sql_dtype = dtype_map.get(dtype.name)
        schema_list.append(f"{column_name} {sql_dtype}")

    query = """CREATE TABLE {} (\n{});""".format(table_name, ',\n'.join(schema_list))

    try:
        execute_query(query, conn)
    except Exception as e:
        print(f"Error: '{e}'")
        conn.rollback()
    print(query)

In [6]:
execute_query('drop table if exists {}'.format(table_name), conn)
create_schema_from_df(df, table_name)

Query successful!

Query successful!

CREATE TABLE traffic (
product_id varchar,
last_visited varchar);


In [7]:
with open(csvpath, "r") as f:
    next(f)
    try:
        cur.copy_from(f, table_name, sep=',', null='')
        conn.commit()

        print("Data inserted into {} using copy_from_datafile() successfully....".format(table_name))
    except Exception as e:
        print(f"Error: '{e}'")
        conn.rollback()

Data inserted into traffic using copy_from_datafile() successfully....


In [9]:
pd.read_sql('select * from {} limit 5'.format(table_name),conn)

c:\Users\jacks\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,product_id,last_visited
0,AH2430,2018-05-19 15:13:00
1,G27341,2018-11-29 16:16:00
2,CM0081,2018-02-01 10:27:00
3,B44832,2018-09-07 20:06:00
4,D98205,2019-07-18 15:26:00


In [10]:
conn.close()